In [1]:
import os
import datetime
import dateutil
import tarfile
import requests
import pandas as pd
import pandas.io.sql as sqlio

%matplotlib inline

In [2]:
os.listdir('../boursorama/2020')

['amsterdam 2020-08-19 11:41:01.953125.bz2',
 'compA 2020-04-13 13:42:01.576208.bz2',
 'compB 2020-07-27 11:01:01.843035.bz2',
 'compB 2020-01-03 15:32:02.016210.bz2',
 'compA 2020-09-21 16:31:01.663943.bz2',
 'compA 2020-11-12 10:01:02.032277.bz2',
 'compB 2020-11-19 10:01:01.563155.bz2',
 'amsterdam 2020-05-04 16:22:01.606666.bz2',
 'compA 2020-07-16 09:11:02.151760.bz2',
 'compB 2020-09-09 10:31:01.513304.bz2',
 'compA 2020-04-17 11:42:01.892034.bz2',
 'amsterdam 2020-07-21 14:41:01.679336.bz2',
 'compB 2020-04-07 13:02:02.334159.bz2',
 'amsterdam 2020-12-18 14:31:01.872836.bz2',
 'amsterdam 2020-04-09 15:12:02.019350.bz2',
 'amsterdam 2020-02-05 17:42:01.716180.bz2',
 'compA 2020-02-20 17:22:02.200629.bz2',
 'amsterdam 2020-07-09 15:21:02.161275.bz2',
 'compB 2020-03-31 11:32:01.507334.bz2',
 'amsterdam 2020-08-26 16:51:02.013325.bz2',
 'compB 2020-10-28 15:11:02.753811.bz2',
 'compA 2020-04-27 15:22:01.595577.bz2',
 'compB 2020-08-06 12:41:02.338692.bz2',
 'compB 2020-03-20 15:42:

In [3]:
df = pd.read_pickle('../boursorama/2020/amsterdam 2020-08-19 11:41:01.953125.bz2')
df

,last,volume,symbol,name
symbol,,,,
AKTX,1.850(c),160468,AKTX,AKARI THERP SP ADR
FLWS,29.980(c),426743,FLWS,1-800-FLOWERS.COM-A
TXG,105.420(c),709891,TXG,10X GENOMICS RG-A
YI,6.770(c),655899,YI,111 SP ADS-A
PIH,4.505(c),382,PIH,1347 PROPT INS H
...,...,...,...,...
ZUMZ,24.050(c),316403,ZUMZ,ZUMIEZ
ZGEN,9.760,0,ZGEN,ZYMOGENETICS INC LGT ZLP
ZYNE,3.870(c),371778,ZYNE,ZYNERBA PHARMA


In [4]:
'''import glob

def collecte_df(directory_path: str = "../boursorama/2019/", market_name: str = "compA") -> pd.DataFrame:
    dfglob = glob.glob1(directory_path, market_name + "*")
    dfs = [pd.read_pickle(directory_path + file) for file in dfglob]
    df = pd.concat(dfs, keys=[file.split()[1] + " " + file.split()[2].split(".bz2")[0] for file in dfglob])
    df.index = pd.to_datetime(df.index.get_level_values(0))
    df = df.rename(columns={'datetime': 'date'})
    return df

'''

import glob
def collecte_df(directory_path:str="../boursorama/2020/", market_name:str="amsterdam") ->pd.DataFrame:
    dfglob = glob.glob1(directory_path, market_name + "*")
    files_path = [directory_path + file for file in dfglob]
    df = pd.concat({dateutil.parser.parse(f.split(market_name)[1].split(".bz2")[0]):pd.read_pickle(f) for f in files_path})
    return df
def convertion(x):
    try:
        
        return float(x)
    except:
        return float(x.replace(' ',''))

def firstFiltering(df, year:str = "2020"):
    df["last"] = [str(x).split("(c)")[0] for x in df["last"]]
    df["last"] = [str(x).split("(s)")[0] for x in df["last"]]
    df['last'] = df['last'].apply(convertion)
    # on retire l'index qui commence par les noms d'entreprises pour finir par leur symbole
    df = df.reset_index(level=1, drop=True)
    df = df.dropna()
    # on nome l'index des date puis on passe la colonne des symboles en index
    df = df.rename_axis('date', axis=0)
    # on retire les jours de congés dont les données ne devraient pas exister
    df.sort_index(inplace=True)
    day_off = [year+'-01-01', year+'-03-30', year+'-04-02', year+'-05-01', year+'-12-25', year+'-12-26']
    for d in day_off:
        if d in df.index:
            df.drop(df.loc[d].index, inplace=True )
    # on passe la colonne des symboles en index et on la met en premier
    #df = df.set_index('symbol', append=True)
    #df = df.swaplevel(0,1).sort_index()
    return df
    

In [5]:
amsterdam = collecte_df(market_name="amsterdam")

In [6]:
amsterdam

last    volume symbol  \
                           symbol                                
2020-08-19 11:41:01.953125 AKTX      1.850(c)    160468   AKTX   
                           FLWS     29.980(c)    426743   FLWS   
                           TXG     105.420(c)    709891    TXG   
                           YI        6.770(c)    655899     YI   
                           PIH       4.505(c)       382    PIH   
...                                       ...       ...    ...   
2020-10-26 12:11:01.817444 CNET      1.510(c)    465775   CNET   
                           ZGEN         9.760         0   ZGEN   
                           ZYNE      3.720(c)    331412   ZYNE   
                           ZYXI     14.470(c)    505732   ZYXI   
                           ZNGA      9.440(c)  15207164   ZNGA   

                                                        name  
                           symbol                             
2020-08-19 11:41:01.953125 AKTX           AKARI THERP SP ADR  
                           FLWS          1-800-FLOWERS.COM-A  
                           TXG             10X GENOMICS RG-A  
                           YI                   111 SP ADS-A  
                           PIH              1347 PROPT INS H  
...                                                      ...  
2020-10-26 12:11:01.817444 CNET               ZW DATA ACTION  
                           ZGEN    ZYMOGENETICS INC  LGT ZLP  
                           ZYNE               ZYNERBA PHARMA  
                           ZYXI                        ZYNEX  
                           ZNGA                   ZYNGA RG-A  

[40738988 rows x 4 columns]

In [ ]:
amsterdam = firstFiltering(amsterdam, year="2020")